In [11]:
import polars as pl
import pycountry

from utils import numeric_iso_col

In [12]:
from geoacled import AcledYear, AcledMonth, clean_column, strip_accents

In [21]:
pycountry.countries.search_fuzzy('Mexico')

[Country(alpha_2='MX', alpha_3='MEX', flag='🇲🇽', name='Mexico', numeric='484', official_name='United Mexican States'),
 Country(alpha_2='US', alpha_3='USA', flag='🇺🇸', name='United States', numeric='840', official_name='United States of America')]

In [ ]:
acled_2022 = AcledYear('Colombia', '2022-01-01', '2023-12-31')

In [ ]:
acled_2022.df
colombia_2022_2023 = acled_2022.df

In [13]:
kei = pl.read_csv('data/kei_6_years.csv')

In [14]:
num_kei = numeric_iso_col(kei, 'REF_AREA').drop_nulls('iso')

STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,FREQ,Frequency of observation,MEASURE,Measure,UNIT_MEASURE,Unit of measure,ACTIVITY,Economic activity,ADJUSTMENT,Adjustment,TRANSFORMATION,Transformation,TIME_PERIOD,Time period,OBS_VALUE,Observation value,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,DECIMALS,Decimals,BASE_PER,Base period,iso
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3098,0,3098,0,0,0,0,0,0,0,3098,0


In [15]:
kei_merge = num_kei['TIME_PERIOD', 'iso', 'OBS_VALUE'].rename({'OBS_VALUE': 'Industrial Production Volume'})

In [16]:
kei_merge

TIME_PERIOD,iso,Industrial Production Volume
str,str,f64
"""2022-07""","""300""",123.369935
"""2022-06""","""300""",122.903508
"""2020-11""","""040""",112.511162
"""2021-06""","""040""",117.511658
"""2021-05""","""040""",117.15448
…,…,…
"""2023-07""","""300""",120.454766
"""2023-05""","""300""",121.504227
"""2023-01""","""300""",116.956564


In [ ]:
def fetch_acled_for_countries(df:pl.DataFrame) -> pl.DataFrame:
    isos = df['iso'].unique().to_list()
    periods = df['TIME_PERIOD'].unique().to_list()
    years = []
    dfs = []
    concat_df = pl.DataFrame()
    print(periods)
    for period in periods:
        years.append(period.split('-')[0])
    years_set = set(years)
    for iso in isos:
        for year in years_set:
            dfs.append(AcledYear(iso=iso, year=year).df)
    return pl.concat(dfs)

In [23]:
test = AcledYear(iso='484', year=2023).df

Query to ACLED: {'format': 'json', 'year': '{2023}', 'iso': '484', 'page': '1'}


In [24]:
test

shape: (0, 0)
┌┐
╞╡
└┘

In [20]:
acled_for_oecd = fetch_acled_for_countries(kei_merge)

['2020-12', '2022-01', '2018-03', '2019-04', '2020-04', '2020-09', '2021-03', '2023-01', '2019-08', '2023-02', '2024-11', '2024-10', '2021-07', '2024-09', '2019-02', '2021-11', '2018-08', '2021-12', '2020-07', '2023-04', '2023-06', '2020-02', '2018-07', '2019-07', '2018-09', '2022-07', '2019-06', '2018-04', '2020-08', '2021-01', '2020-05', '2021-06', '2024-05', '2023-10', '2019-05', '2021-09', '2023-03', '2018-10', '2023-11', '2018-11', '2018-02', '2020-06', '2021-04', '2018-06', '2020-01', '2020-10', '2021-08', '2022-05', '2022-04', '2018-12', '2021-02', '2024-04', '2022-06', '2024-03', '2019-01', '2024-12', '2023-08', '2018-01', '2022-09', '2019-09', '2019-12', '2024-02', '2024-06', '2022-11', '2020-11', '2020-03', '2022-02', '2019-10', '2024-07', '2024-01', '2024-08', '2022-08', '2022-03', '2019-03', '2019-11', '2022-12', '2023-12', '2021-10', '2018-05', '2022-10', '2023-05', '2021-05', '2023-07', '2023-09']
Query to ACLED: {'format': 'json', 'year': "{'2020'}", 'iso': '428', 'page'

KeyboardInterrupt: 

In [7]:
kei.head()

STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,FREQ,Frequency of observation,MEASURE,Measure,UNIT_MEASURE,Unit of measure,ACTIVITY,Economic activity,ADJUSTMENT,Adjustment,TRANSFORMATION,Transformation,TIME_PERIOD,Time period,OBS_VALUE,Observation value,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,DECIMALS,Decimals,BASE_PER,Base period
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,i64,str,i64,str,i64,str
"""DATAFLOW""","""OECD.SDD.STES:DSD_KEI@DF_KEI(4…","""Key short-term economic indica…","""I""","""LTU""","""Lithuania""","""M""","""Monthly""","""PRVM""","""Production volume""","""IX""","""Index""","""BTE""","""Industry (except construction)""","""Y""","""Calendar and seasonally adjust…","""_Z""","""Not applicable""","""2025-09""",null,150.429799,null,"""A""","""Normal value""",0,"""Units""",1,"""One""",2015,null
"""DATAFLOW""","""OECD.SDD.STES:DSD_KEI@DF_KEI(4…","""Key short-term economic indica…","""I""","""LTU""","""Lithuania""","""M""","""Monthly""","""PRVM""","""Production volume""","""IX""","""Index""","""BTE""","""Industry (except construction)""","""Y""","""Calendar and seasonally adjust…","""_Z""","""Not applicable""","""2025-06""",null,151.432665,null,"""A""","""Normal value""",0,"""Units""",1,"""One""",2015,null
"""DATAFLOW""","""OECD.SDD.STES:DSD_KEI@DF_KEI(4…","""Key short-term economic indica…","""I""","""BEL""","""Belgium""","""M""","""Monthly""","""PRVM""","""Production volume""","""IX""","""Index""","""BTE""","""Industry (except construction)""","""Y""","""Calendar and seasonally adjust…","""_Z""","""Not applicable""","""2025-07""",null,128.724015,null,"""A""","""Normal value""",0,"""Units""",1,"""One""",2015,null
"""DATAFLOW""","""OECD.SDD.STES:DSD_KEI@DF_KEI(4…","""Key short-term economic indica…","""I""","""BEL""","""Belgium""","""M""","""Monthly""","""PRVM""","""Production volume""","""IX""","""Index""","""BTE""","""Industry (except construction)""","""Y""","""Calendar and seasonally adjust…","""_Z""","""Not applicable""","""2025-08""",null,126.932353,null,"""A""","""Normal value""",0,"""Units""",1,"""One""",2015,null
"""DATAFLOW""","""OECD.SDD.STES:DSD_KEI@DF_KEI(4…","""Key short-term economic indica…","""I""","""BEL""","""Belgium""","""M""","""Monthly""","""PRVM""","""Production volume""","""IX""","""Index""","""BTE""","""Industry (except construction)""","""Y""","""Calendar and seasonally adjust…","""_Z""","""Not applicable""","""2025-09""",null,125.967612,null,"""A""","""Normal value""",0,"""Units""",1,"""One""",2015,null


In [ ]:
acled = pl.read_csv('data/acled_south_america_18-23.csv')
acled.describe()

In [ ]:
def join_on_acled(df: pl.DataFrame,
                  acled_df: pl.DataFrame,
                  ref_col: str,
                  val_col: str,
                  date_col: str):
    df = df.with_columns(pl.col([ref_col, val_col]))

In [ ]:
acled_year_month = acled.with_columns(
    pl.col('event_date')
    .str.to_date()
    .dt.strftime("%Y-%m")
    .alias('year_month'))
acled_grouped = acled_year_month.select(['year_month',
                                         'country'])
data = acled_grouped.group_by(['year_month',
                               'country']).len()

In [ ]:
data['country'].unique()

In [ ]:
countries = data['country'].unique().to_list()

In [ ]:
lst = kei['Reference area'].unique().to_list()

In [ ]:
print(lst)

In [ ]:
oecd_southcom = kei.filter(pl.col('Reference area').is_in(countries))

In [ ]:
oecd_southcom.group_by(['TIME_PERIOD', 'Reference area']).len()

In [ ]:
oecd_southcom['Reference area'].unique().len()

In [ ]:
google_mob = pl.read_csv('data/Global_Mobility_Report.csv')

In [ ]:
google_mob.describe()